In [1]:
import json
import numpy as np
import os
import sys
import os.path as osp
import shutil
import subprocess as sp
%autosave 5

Autosaving every 5 seconds


## Prepare folder

In [2]:
fMRI_data_dir = '/media/brain/Spatial/spatialfMRI/fMRI_data/'
sub_range = range(18, 19)

In [3]:
for sub_id in sub_range:
    os.makedirs(fMRI_data_dir+'spatialfMRI_nii/sub-{:02d}/anat'.format(sub_id),exist_ok=True)
    os.makedirs(fMRI_data_dir+'spatialfMRI_nii/sub-{:02d}/func'.format(sub_id),exist_ok=True)
    os.makedirs(fMRI_data_dir+'spatialfMRI_nii/sub-{:02d}/fmap'.format(sub_id),exist_ok=True)


## Install dcm2niix
- dcm2nii DICOM to NIfTI converter: compiled versions available from NITRC.
- https://github.com/rordenlab/dcm2niix
- Using the updated dcm2niix instead of dcm2niix_afni avoids the problem of naming for ME data, that one nifti file does not get a suffix indicating the echo index 
Then we move that tmp folder to under each subject's BIDS folder created above

In [ ]:
!sudo apt-get -y install dcm2niix

In [ ]:
!curl -fLO https://github.com/rordenlab/dcm2niix/releases/latest/download/dcm2niix_lnx.zip
!unzip dcm2niix_lnx.zip

## dcm2niix convert
- Now we should use dcm2niix to convert dicom files and place in tmp folder under each subject
- example command line:
dcm2niix -f tmp/%f_%p_%s -z y ../dcm_ME_benchmark/ME_benchmark_007-0930-1524/dcm/

In [4]:
%%bash 

subject="sub-18"
./dcm2niix -b y -f tmp/%f_%p_%s_%e -z y /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_dicom/${subject}


Chris Rorden's dcm2niiX version v1.0.20240202  (JP2:OpenJPEG) (JP-LS:CharLS) GCC8.4.0 x86-64 (64-bit Linux)
Found 32373 DICOM file(s)
Slices not stacked: echo varies (TE 96.53, 41.51; echo 4, 2). Use 'merge 2D slices' option to force stacking
Convert 225 DICOM as /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_dicom/sub-18/tmp/sub-18_multi_echo_2p8_1500_MB5_TE13p8_34_4_ph (70x70x45x225)
Convert 225 DICOM as /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_dicom/sub-18/tmp/sub-18_multi_echo_2p8_1500_MB5_TE13p8_34_2_ph (70x70x45x225)
Convert 225 DICOM as /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_dicom/sub-18/tmp/sub-18_multi_echo_2p8_1500_MB5_TE13p8_34_5_ph (70x70x45x225)
Convert 225 DICOM as /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_dicom/sub-18/tmp/sub-18_multi_echo_2p8_1500_MB5_TE13p8_34_3_ph (70x70x45x225)
Convert 225 DICOM as /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_dicom/sub-18/tmp/sub-18_multi_echo_2p8_1500_MB5_TE13p8_34_1_ph (7

## Process anatomical images

In [4]:
# normalized structure
for sub_id in sub_range:
    in_niis = [fMRI_data_dir+'spatialfMRI_dicom/sub-{sub:02d}/tmp/sub-{sub:02d}_T1w_MPR_51_1.nii.gz'.format(sub=sub_id),
               fMRI_data_dir+'spatialfMRI_dicom/sub-{sub:02d}/tmp/dcm_T1w_MPR_51_1.nii.gz'.format(sub=sub_id)]
    in_jsons = [fMRI_data_dir+'spatialfMRI_dicom/sub-{sub:02d}/tmp/sub-{sub:02d}_T1w_MPR_51_1.json'.format(sub=sub_id),
               fMRI_data_dir+'spatialfMRI_dicom/sub-{sub:02d}/tmp/dcm_T1w_MPR_51_1.json'.format(sub=sub_id)]
    out_nii = fMRI_data_dir+'spatialfMRI_nii/sub-{sub:02d}/anat/sub-{sub:02d}_T1w.nii.gz'.format(sub=sub_id)
    out_json = fMRI_data_dir+'spatialfMRI_nii/sub-{sub:02d}/anat/sub-{sub:02d}_T1w.json'.format(sub=sub_id)
    for in_nii in in_niis:
        if osp.isfile(in_nii):
            shutil.move(in_nii, out_nii)
            print(in_nii, ' moved to ', out_nii)
    for in_json in in_jsons:
        if osp.isfile(in_json):
            shutil.move(in_json, out_json)
            print(in_json, ' moved to ', out_json)

/media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_dicom/sub-18/tmp/sub-18_T1w_MPR_51_1.nii.gz  moved to  /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/anat/sub-18_T1w.nii.gz
/media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_dicom/sub-18/tmp/sub-18_T1w_MPR_51_1.json  moved to  /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/anat/sub-18_T1w.json


## Process func images

Prepare a dictionary mapping from task and sequence to the run id for each subject

In [6]:
# singleband as reference
subject = 'sub-18'
sbref_dict = {
    subject: {
        'run-00': {
            'ME5': 6,
        },
        'run-01': {
            'ME5': 9,
        },
        'run-02': {
            'ME5': 12,
        },
        'run-03': {
            'ME5': 15,
        },
        'run-04': {
            'ME5': 18,
        },
        'run-05': {
            'ME5': 21,
        },
        'run-06': {
            'ME5': 24,
        },
        'run-07': {
            'ME5': 29,
        },
        'run-08': {
            'ME5': 32,
        },
        'run-09': {
            'ME5': 35,
        },
        'run-10': {
            'ME5': 38,
        },
        'run-11': {
            'ME5': 41,
        },
        'run-12': {
            'ME5': 44,
        },
        'run-13': {
            'ME5': 47,
        },
    },
}
# multiband bold
bold_dict = {
    subject: {
        'run-00': {
            'ME5': 7,
        },
        'run-01': {
            'ME5': 10,
        },
        'run-02': {
            'ME5': 13,
        },
        'run-03': {
            'ME5': 16,
        },
        'run-04': {
            'ME5': 19,
        },
        'run-05': {
            'ME5': 22,
        },
        'run-06': {
            'ME5': 25,
        },
        'run-07': {
            'ME5': 30,
        },
        'run-08': {
            'ME5': 33,
        },
        'run-09': {
            'ME5': 36,
        },
        'run-10': {
            'ME5': 39,
        },
        'run-11': {
            'ME5': 42,
        },
        'run-12': {
            'ME5': 45,
        },
        'run-13': {
            'ME5': 48,
        },
    },
}
# multiband phase
ph_dict = {
    subject: {
        'run-00': {
            'ME5': 8,
        },
        'run-01': {
            'ME5': 11,
        },
        'run-02': {
            'ME5': 14,
        },
        'run-03': {
            'ME5': 17,
        },
        'run-04': {
            'ME5': 20,
        },
        'run-05': {
            'ME5': 23,
        },
        'run-06': {
            'ME5': 26,
        },
        'run-07': {
            'ME5': 31,
        },
        'run-08': {
            'ME5': 34,
        },
        'run-09': {
            'ME5': 37,
        },
        'run-10': {
            'ME5': 40,
        },
        'run-11': {
            'ME5': 43,
        },
        'run-12': {
            'ME5': 46,
        },
        'run-13': {
            'ME5': 49,
        },
    },
}

In [7]:
def move_run_me(sub_id, task, run, scanner_run_id, modality, n_echo, suffix):
    echo_notfound = [] # Occasionally dcm2niix_afni omits the _e? extension in multi-echo files
    
    all_files = os.listdir(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id))

    normal_files = [file for file in all_files  for echo in range(1, n_echo + 1) if '_{}_'.format(scanner_run_id) in file
             and file.endswith('_{}.{}'.format(echo, suffix))]

    abnormal_files = [file for file in all_files if file.endswith('_{}.{}'.format(scanner_run_id, suffix)) ]
    if len(abnormal_files) == 0 and len(normal_files) == n_echo:
        for echo in range(1, n_echo + 1):
            file = [file for file in normal_files if '_{}_{}.{}'.format(scanner_run_id, echo, suffix) in file]
            os.rename(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id)
                       + file[0],
                        fMRI_data_dir+'spatialfMRI_nii/{subj}/func/{subj}_task-{task}_{run}_echo-{echo}_{modality}.{suffix}'.format(
                        subj=sub_id, task=task, run=run, echo=echo, modality=modality, suffix=suffix))
    elif len(abnormal_files) == 1 and len(normal_files) == n_echo - 1:
        for echo in range(1, n_echo + 1):
            file = [file for file in normal_files if '_{}_{}.{}'.format(scanner_run_id, echo, suffix) in file]
        
            if len(file) == 1:
                os.rename(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id)
                           + file[0],
                            fMRI_data_dir+'spatialfMRI_nii/{subj}/func/{subj}_task-{task}_{run}_echo-{echo}_{modality}.{suffix}'.format(
                            subj=sub_id, task=task, run=run, echo=echo, modality=modality, suffix=suffix))
            elif len(file) == 0:
                echo_notfound.append(echo)
        os.rename(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id)
                   + abnormal_files[0],
                    fMRI_data_dir+'spatialfMRI_nii/{subj}/func/{subj}_task-{task}_{run}_echo-{echo}_{modality}.{suffix}'.format(
                    subj=sub_id, task=task, run=run, echo=echo_notfound[0], modality=modality, suffix=suffix))
    else:
        print(sub_id, ', modality:', modality)
        print("move_run_me() don't know how to deal with these files:")
        print(normal_files, abnormal_files)

def move_run_ph(sub_id, task, run, scanner_run_id, modality, n_echo, suffix):
    echo_notfound = [] # Occasionally dcm2niix_afni omits the _e? extension in multi-echo files

    global all_files, normal_files, abnormal_files
    
    all_files = os.listdir(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id))

    normal_files = [file for file in all_files  for echo in range(1, n_echo + 1) if '_{}_'.format(scanner_run_id) in file
             and file.endswith('_{}_ph.{}'.format(echo, suffix))]
             
    abnormal_files = [file for file in all_files if file.endswith('_{}_ph.{}'.format(scanner_run_id, suffix)) ]
    if len(abnormal_files) == 0 and len(normal_files) == n_echo:
        for echo in range(1, n_echo + 1):
            file = [file for file in normal_files if '_{}_{}_ph.{}'.format(scanner_run_id, echo, suffix) in file]
            os.rename(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id)
                       + file[0],
                        fMRI_data_dir+'spatialfMRI_nii/{subj}/func/{subj}_task-{task}_{run}_echo-{echo}_{modality}.{suffix}'.format(
                        subj=sub_id, task=task, run=run, echo=echo, modality=modality, suffix=suffix))
    elif len(abnormal_files) == 1 and len(normal_files) == n_echo - 1:
        for echo in range(1, n_echo + 1):
            file = [file for file in normal_files if '_{}_{}_ph.{}'.format(scanner_run_id, echo, suffix) in file]
        
            if len(file) == 1:
                os.rename(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id)
                           + file[0],
                            fMRI_data_dir+'spatialfMRI_nii/{subj}/func/{subj}_task-{task}_{run}_echo-{echo}_{modality}.{suffix}'.format(
                            subj=sub_id, task=task, run=run, echo=echo, modality=modality, suffix=suffix))
            elif len(file) == 0:
                echo_notfound.append(echo)
        os.rename(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id)
                   + abnormal_files[0],
                    fMRI_data_dir+'spatialfMRI_nii/{subj}/func/{subj}_task-{task}_{run}_echo-{echo}_{modality}.{suffix}'.format(
                    subj=sub_id, task=task, run=run, echo=echo, modality=modality, suffix=suffix))
    else:
        print(sub_id, ', modality:', modality)
        print("move_run_ph() don't know how to deal with these files:")
        print(normal_files, abnormal_files)


In [8]:
# sub_range = range(1, 2) #[0,1]
n_echo = 5
task = "geo"

# singleband as reference with multi-echo
modality = 'part-mag_sbref'
for sub in sub_range:
    for run in range(0, 14):
        for acq in ['ME5']:
            sub_id = 'sub-{:02d}'.format(sub)
            run_id = 'run-{:02d}'.format(run)
            assert sbref_dict[sub_id][run_id][acq] != bold_dict[sub_id][run_id][acq], \
                'SBref image is the same as BOLD. Please double-check'
            scanner_run_id = sbref_dict[sub_id][run_id][acq]
            move_run_me(sub_id, task, run_id, scanner_run_id, modality, n_echo, 'nii.gz')
            move_run_me(sub_id, task, run_id, scanner_run_id, modality, n_echo, 'json')

# singleband as reference with multi-echo
modality = 'part-phase_sbref'
for sub in sub_range:
    for run in range(0, 14):
        for acq in ['ME5']:
            sub_id = 'sub-{:02d}'.format(sub)
            run_id = 'run-{:02d}'.format(run)
            assert sbref_dict[sub_id][run_id][acq] != bold_dict[sub_id][run_id][acq], \
                'SBref image is the same as BOLD. Please double-check'
            scanner_run_id = sbref_dict[sub_id][run_id][acq]
            move_run_ph(sub_id, task, run_id, scanner_run_id, modality, n_echo, 'nii.gz')
            move_run_ph(sub_id, task, run_id, scanner_run_id, modality, n_echo, 'json')

# multiband bold with multi-echo
modality = 'part-mag_bold'
for sub in sub_range:
    for run in range(0, 14):
        for acq in ['ME5']:
            sub_id = 'sub-{:02d}'.format(sub)
            run_id = 'run-{:02d}'.format(run)
            assert sbref_dict[sub_id][run_id][acq] != bold_dict[sub_id][run_id][acq], \
                'SBref image is the same as BOLD. Please double-check'
            scanner_run_id = bold_dict[sub_id][run_id][acq]
            move_run_me(sub_id, task, run_id, scanner_run_id, modality, n_echo, 'nii.gz')
            move_run_me(sub_id, task, run_id, scanner_run_id, modality, n_echo, 'json')

# multiband phase  with multi-echo
modality = 'part-phase_bold'
for sub in sub_range:
    for run in range(0, 14):
        for acq in ['ME5']:
            sub_id = 'sub-{:02d}'.format(sub)
            run_id = 'run-{:02d}'.format(run)
            assert ph_dict[sub_id][run_id][acq] != bold_dict[sub_id][run_id][acq], \
                'phase image is the same as BOLD. Please double-check'
            assert ph_dict[sub_id][run_id][acq] != sbref_dict[sub_id][run_id][acq], \
                'phase image is the same as sbref. Please double-check'
            scanner_run_id = ph_dict[sub_id][run_id][acq]
            move_run_ph(sub_id, task, run_id, scanner_run_id, modality, n_echo, 'nii.gz')
            move_run_ph(sub_id, task, run_id, scanner_run_id, modality, n_echo, 'json')

# Add TaskName field. 
It is required to have a field named TaskName in the json file. So the following function fixes this

In [9]:
def fix_json_TaskName(file, TaskName, TaskDescription=None, CogAtlasID=None):
    with open(file) as json_file:
        json_decoded = json.load(json_file)
        json_decoded['TaskName'] = TaskName
        if not TaskDescription is None:
            json_decoded['TaskDescription'] = TaskDescription
        if not CogAtlasID is None:
            json_decoded['CogAtlasID'] = CogAtlasID
#         print(json_decoded)
            
    with open(file, 'w') as json_file:
        json.dump(json_decoded, json_file, indent=8)

In [10]:
TaskDescriptions = {

    'run-00': 'In the experiments, subjects watch a movie which may contain'\
        'views in a city or in the countryside as if they are sitting in a car.'\
        'They are asked to make simple judgments based on pictures that they'\
        'see using a button.',
    
    'run-01': 'In the experiments, subjects watch a movie which may contain'\
        'views in a city or in the countryside as if they are sitting in a car.'\
        'They are asked to make simple judgments based on pictures that they'\
        'see using a button.',
    
    'run-02': 'In the experiments, subjects watch a movie which may contain'\
        'views in a city or in the countryside as if they are sitting in a car.'\
        'They are asked to make simple judgments based on pictures that they'\
        'see using a button.',
        
    'run-03': 'In the experiments, subjects watch a movie which may contain'\
        'views in a city or in the countryside as if they are sitting in a car.'\
        'They are asked to make simple judgments based on pictures that they'\
        'see using a button.',

    'run-04': 'In the experiments, subjects watch a movie which may contain'\
        'views in a city or in the countryside as if they are sitting in a car.'\
        'They are asked to make simple judgments based on pictures that they'\
        'see using a button.',

    'run-05': 'In the experiments, subjects watch a movie which may contain'\
        'views in a city or in the countryside as if they are sitting in a car.'\
        'They are asked to make simple judgments based on pictures that they'\
        'see using a button.',

    'run-06': 'In the experiments, subjects watch a movie which may contain'\
        'views in a city or in the countryside as if they are sitting in a car.'\
        'They are asked to make simple judgments based on pictures that they'\
        'see using a button.',

    'run-07': 'In the experiments, subjects watch a movie which may contain'\
        'views in a city or in the countryside as if they are sitting in a car.'\
        'They are asked to make simple judgments based on pictures that they'\
        'see using a button.',
    
    'run-08': 'In the experiments, subjects watch a movie which may contain'\
        'views in a city or in the countryside as if they are sitting in a car.'\
        'They are asked to make simple judgments based on pictures that they'\
        'see using a button.',
    
    'run-09': 'In the experiments, subjects watch a movie which may contain'\
        'views in a city or in the countryside as if they are sitting in a car.'\
        'They are asked to make simple judgments based on pictures that they'\
        'see using a button.',
        
    'run-10': 'In the experiments, subjects watch a movie which may contain'\
        'views in a city or in the countryside as if they are sitting in a car.'\
        'They are asked to make simple judgments based on pictures that they'\
        'see using a button.',

    'run-11': 'In the experiments, subjects watch a movie which may contain'\
        'views in a city or in the countryside as if they are sitting in a car.'\
        'They are asked to make simple judgments based on pictures that they'\
        'see using a button.',

    'run-12': 'In the experiments, subjects watch a movie which may contain'\
        'views in a city or in the countryside as if they are sitting in a car.'\
        'They are asked to make simple judgments based on pictures that they'\
        'see using a button.',

    'run-13': 'In the experiments, subjects watch a movie which may contain'\
        'views in a city or in the countryside as if they are sitting in a car.'\
        'They are asked to make simple judgments based on pictures that they'\
        'see using a button.',
}

CogAtlasIDs = {
    'run-00': 'http://www.cognitiveatlas.org/task/id/trm_4f241173868a3/',
    'run-01': 'http://www.cognitiveatlas.org/task/id/trm_4f241173868a3/',
    'run-02': 'http://www.cognitiveatlas.org/task/id/trm_4f241173868a3/',
    'run-03': 'http://www.cognitiveatlas.org/task/id/trm_4f241173868a3/',
    'run-04': 'http://www.cognitiveatlas.org/task/id/trm_4f241173868a3/',
    'run-05': 'http://www.cognitiveatlas.org/task/id/trm_4f241173868a3/',
    'run-06': 'http://www.cognitiveatlas.org/task/id/trm_4f241173868a3/',
    'run-07': 'http://www.cognitiveatlas.org/task/id/trm_4f241173868a3/',
    'run-08': 'http://www.cognitiveatlas.org/task/id/trm_4f241173868a3/',
    'run-09': 'http://www.cognitiveatlas.org/task/id/trm_4f241173868a3/',
    'run-10': 'http://www.cognitiveatlas.org/task/id/trm_4f241173868a3/',
    'run-11': 'http://www.cognitiveatlas.org/task/id/trm_4f241173868a3/',
    'run-12': 'http://www.cognitiveatlas.org/task/id/trm_4f241173868a3/',
    'run-13': 'http://www.cognitiveatlas.org/task/id/trm_4f241173868a3/',
}

for sub in sub_range:
    sub_id = 'sub-{:02d}'.format(sub)
    for run in range(0, 14):
        run_id = 'run-{:02d}'.format(run)
        for modality in ['part-mag_sbref', 'part-phase_sbref', 'part-mag_bold', 'part-phase_bold']:
            for acq in ['ME5']:
                for echo in range(1, n_echo + 1):
                
                    file = fMRI_data_dir+'spatialfMRI_nii/{sub}/func/{sub}_task-{task}_{run_id}_echo-{echo}_{mod}.json'.format(
                        sub=sub_id, task=task, run_id=run_id, echo=echo, mod=modality)
                    if osp.isfile(file):
                        fix_json_TaskName(file, run_id, TaskDescriptions[run_id], CogAtlasIDs[run_id])
                        print(file, ' is fixed')


/media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-1_part-mag_sbref.json  is fixed
/media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-2_part-mag_sbref.json  is fixed
/media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-3_part-mag_sbref.json  is fixed
/media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-4_part-mag_sbref.json  is fixed
/media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-5_part-mag_sbref.json  is fixed
/media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-1_part-phase_sbref.json  is fixed
/media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-2_part-phase_sbref.json  is fixed
/media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_t

## Work on SpinEcho data

In [11]:
SpinEcho_dict = {
    'sub-18': {
        'ME5': [
            {
                'scanner_run_id': [27, 28],
                'intendedfor': [
                    "task-geo_run-00_echo-{}_part-mag_sbref",
                    "task-geo_run-00_echo-{}_part-mag_bold",
                    "task-geo_run-00_echo-{}_part-phase_bold",
                    "task-geo_run-01_echo-{}_part-mag_sbref",
                    "task-geo_run-01_echo-{}_part-mag_bold",
                    "task-geo_run-01_echo-{}_part-phase_bold",
                    "task-geo_run-02_echo-{}_part-mag_sbref",
                    "task-geo_run-02_echo-{}_part-mag_bold",
                    "task-geo_run-02_echo-{}_part-phase_bold",
                    "task-geo_run-03_echo-{}_part-mag_sbref",
                    "task-geo_run-03_echo-{}_part-mag_bold",
                    "task-geo_run-03_echo-{}_part-phase_bold",
                    "task-geo_run-04_echo-{}_part-mag_sbref",
                    "task-geo_run-04_echo-{}_part-mag_bold",
                    "task-geo_run-04_echo-{}_part-phase_bold",
                    "task-geo_run-05_echo-{}_part-mag_sbref",
                    "task-geo_run-05_echo-{}_part-mag_bold",
                    "task-geo_run-05_echo-{}_part-phase_bold",
                    "task-geo_run-06_echo-{}_part-mag_sbref",
                    "task-geo_run-06_echo-{}_part-mag_bold",
                    "task-geo_run-06_echo-{}_part-phase_bold",
                    "task-geo_run-07_echo-{}_part-mag_sbref",
                    "task-geo_run-07_echo-{}_part-mag_bold",
                    "task-geo_run-07_echo-{}_part-phase_bold",
                    "task-geo_run-08_echo-{}_part-mag_sbref",
                    "task-geo_run-08_echo-{}_part-mag_bold",
                    "task-geo_run-08_echo-{}_part-phase_bold",
                    "task-geo_run-09_echo-{}_part-mag_sbref",
                    "task-geo_run-09_echo-{}_part-mag_bold",
                    "task-geo_run-09_echo-{}_part-phase_bold",
                    "task-geo_run-10_echo-{}_part-mag_sbref",
                    "task-geo_run-10_echo-{}_part-mag_bold",
                    "task-geo_run-10_echo-{}_part-phase_bold",
                    "task-geo_run-11_echo-{}_part-mag_sbref",
                    "task-geo_run-11_echo-{}_part-mag_bold",
                    "task-geo_run-11_echo-{}_part-phase_bold",
                    "task-geo_run-12_echo-{}_part-mag_sbref",
                    "task-geo_run-12_echo-{}_part-mag_bold",
                    "task-geo_run-12_echo-{}_part-phase_bold",
                    "task-geo_run-13_echo-{}_part-mag_sbref",
                    "task-geo_run-13_echo-{}_part-mag_bold",
                    "task-geo_run-13_echo-{}_part-phase_bold",
                ]
            }
        ],
    },
}

In [12]:
def move_SpinEcho_me(sub_id, acq, scanner_run_id, n_echo):

    global all_files, nii_files, json_files
    all_files = os.listdir(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id))
    nii_files = [file for file in all_files for run in scanner_run_id if 'SpinEcho' in file
                 and file.endswith('{}_1.nii.gz'.format(run))]
    json_files = [file for file in all_files for run in scanner_run_id if 'SpinEcho' in file
                 and file.endswith('{}_1.json'.format(run))]
    
    if len(nii_files) == 2 and len(json_files) == 2:
        AP_nii_file = [file for file in nii_files if '_AP_' in file][0]
        PA_nii_file = [file for file in nii_files if '_PA_' in file][0]
        AP_json_file = [file for file in json_files if '_AP_' in file][0]
        PA_json_file = [file for file in json_files if '_PA_' in file][0]
        
        with open(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id) + AP_json_file) as json_file:
            json_decoded = json.load(json_file)
            json_decoded['IntendedFor'] = [
                'func/{sub_id}_{task}.nii.gz'.format(sub_id=sub_id, task=task.format(echo))
                for echo in range(1, n_echo + 1) for task in SpinEcho_dict[sub_id][acq][0]['intendedfor']]
        with open(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id) + AP_json_file, 'w') as json_file:
            json.dump(json_decoded, json_file, indent=8)
            
        with open(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id) + PA_json_file) as json_file:
            json_decoded = json.load(json_file)
            json_decoded['IntendedFor'] = [
                'func/{sub_id}_{task}.nii.gz'.format(sub_id=sub_id, task=task.format(echo))
                for echo in range(1, n_echo + 1) for task in SpinEcho_dict[sub_id][acq][0]['intendedfor']]
        with open(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id) + PA_json_file, 'w') as json_file:
            json.dump(json_decoded, json_file, indent=8)
                
#             print(AP_json_decoded)
        
        shutil.move(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id)
                   + AP_nii_file,
                    fMRI_data_dir+'spatialfMRI_nii/{subj}/fmap/{subj}_acq-{acq}_dir-AP_epi.nii.gz'.format(
                    subj=sub_id, acq=acq))
        shutil.move(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id)
                   + PA_nii_file,
                    fMRI_data_dir+'spatialfMRI_nii/{subj}/fmap/{subj}_acq-{acq}_dir-PA_epi.nii.gz'.format(
                    subj=sub_id, acq=acq))
        
        shutil.move(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id)
                   + AP_json_file,
                    fMRI_data_dir+'spatialfMRI_nii/{subj}/fmap/{subj}_acq-{acq}_dir-AP_epi.json'.format(
                    subj=sub_id, acq=acq))
        shutil.move(fMRI_data_dir+'spatialfMRI_dicom/{}/tmp/'.format(sub_id)
                   + PA_json_file,
                    fMRI_data_dir+'spatialfMRI_nii/{subj}/fmap/{subj}_acq-{acq}_dir-PA_epi.json'.format(
                    subj=sub_id, acq=acq))

    else:
        print("I don't know how to deal with these files:")
        print(nii_files)
        print(json_files)

In [13]:
# sub_range = range(1, 2)
n_echo = 5
for sub in sub_range:
    for acq in ['ME5']:
        sub_id = 'sub-{:02d}'.format(sub)
        scanner_run_id = SpinEcho_dict[sub_id][acq][0]['scanner_run_id']
        move_SpinEcho_me(sub_id, acq, scanner_run_id, n_echo)


In [14]:
# sp.getoutput('3dinfo -nv ../data/Nifti_ME_benchmark/sub-002/func/sub-002_task-visual_acq-ME5_echo-5_bold.nii.gz')

## Fix the problem of unequal number of volumes across TEs

In [15]:
# sub_range = range(0, 1)
n_echo = 5
os.environ['AFNI_NIFTI_TYPE_WARN']='NO'
for sub in sub_range:
    for acq in ['ME5']:
        sub_id = 'sub-{:02d}'.format(sub)
        for run in range(0, 14):
            run_id = 'run-{:02d}'.format(run)
            for modality in ['part-mag_sbref', 'part-phase_sbref', 'part-mag_bold', 'part-phase_bold']:
                n_volume = []
                for echo in range(1, n_echo + 1):
                    filename = fMRI_data_dir+'spatialfMRI_nii/{sub_id}/func/{sub_id}_task-{task}_{run_id}_echo-{echo}_{modality}.nii.gz'.format(
                        sub_id=sub_id, task=task, run_id=run_id, echo=echo, modality=modality)
                    if osp.isfile(filename):
                        n_volume.append(int(sp.getoutput('3dinfo -nv ' + filename)))
                if np.unique(n_volume).size > 1 and len(n_volume) == n_echo:
                    print(n_volume)
                    n_keep = np.min(n_volume)
                    print(n_keep)
                    for echo in range(1, n_echo + 1):
                        filename = fMRI_data_dir+'spatialfMRI_nii/{sub_id}/func/{sub_id}_task-{task}_{run_id}_echo-{echo}_{modality}.nii.gz'.format(
                            sub_id=sub_id, task=task, run_id=run_id, echo=echo, modality=modality)
                        new_filename = filename.replace('.nii.gz', '_fixed.nii.gz')
                        command = "3dTcat -prefix " + new_filename + ' ' + filename + "'[0..{}]'".format(n_keep - 1)
                        print(command)
                        os.system(command)

                        command = 'mv ' + new_filename + ' ' + filename
                        print(command)
                        os.system(command)

## Fix the problem of unequal number of volumes across runs

In [16]:
# time duration: 4(fixation) + 5m*60(video) + 10(fixation) + (3(Q)+1.5(A)+0.5(F))*2 + 10(fixation) = 334s
# length of TR: 334/1.5=222.7≈223 TR
# set the max is 223 TR
n_keep = 223
n_echo = 5
os.environ['AFNI_NIFTI_TYPE_WARN']='NO'
for sub in sub_range:
    for acq in ['ME5']:
        sub_id = 'sub-{:02d}'.format(sub)
        for run in range(0, 14):
            run_id = 'run-{:02d}'.format(run)
            n_volume = []
            for modality in ['part-mag_sbref', 'part-phase_sbref', 'part-mag_bold', 'part-phase_bold']:
                for echo in range(1, n_echo + 1):
                    filename = fMRI_data_dir+'spatialfMRI_nii/{sub_id}/func/{sub_id}_task-{task}_{run_id}_echo-{echo}_{modality}.nii.gz'.format(
                        sub_id=sub_id, task=task, run_id=run_id, echo=echo, modality=modality)
                    if osp.isfile(filename):
                        num_tr = int(sp.getoutput('3dinfo -nv ' + filename))
                        if num_tr == 1: continue
                        # n_volume.append(int(sp.getoutput('3dinfo -nv ' + filename)))

                    new_filename = filename.replace('.nii.gz', '_fixed.nii.gz')
                    command = "3dTcat -prefix " + new_filename + ' ' + filename + "'[0..{}]'".format(n_keep - 1)
                    # print(command)
                    os.system(command)

                    command = 'mv ' + new_filename + ' ' + filename
                    # print(command)
                    os.system(command)


++ 3dTcat: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ elapsed time = 25.1 s
++ 3dTcat: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ elapsed time = 23.3 s
++ 3dTcat: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ elapsed time = 22.2 s
++ 3dTcat: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ elapsed time = 22.0 s
++ 3dTcat: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ elapsed time = 22.9 s
++ 3dTcat: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ elapsed time = 35.6 s
++ 3dTcat: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ elapsed time = 36.7 s
++ 3dTcat: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ elapsed time = 37.0 s
++ 3dTcat: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ elapsed time = 37.0 s
++ 3dTcat: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ elapsed time = 37.2 s
++ 3dTcat: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ elapsed time = 25.0 s
++ 3dTcat: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ el

# Add slice timing

In [17]:
import json
import numpy as np
import os
import sys
import os.path as osp
import shutil
import subprocess
import json
%autosave 5

Autosaving every 5 seconds


In [18]:
n_echo = 5

os.environ['AFNI_NIFTI_TYPE_WARN']='NO'
for sub in sub_range:
    for acq in ['ME5']:
        sub_id = 'sub-{:02d}'.format(sub)
        for run in range(0, 14):
            run_id = 'run-{:02d}'.format(run)
            for modality in ['part-mag_sbref', 'part-phase_sbref', 'part-mag_bold', 'part-phase_bold']:
                n_volume = []
                for echo in range(1, n_echo + 1):
                    file_dir = fMRI_data_dir+'spatialfMRI_nii/{sub_id}/func/'.format(sub_id=sub_id)
                    json_filename = fMRI_data_dir+'spatialfMRI_nii/{sub_id}/func/{sub_id}_task-{task}_{run_id}_echo-{echo}_{modality}.json'.format(
                        sub_id=sub_id, task=task, run_id=run_id, echo=echo, modality=modality)
                    nii_filename = fMRI_data_dir+'spatialfMRI_nii/{sub_id}/func/{sub_id}_task-{task}_{run_id}_echo-{echo}_{modality}.nii.gz'.format(
                        sub_id=sub_id, task=task, run_id=run_id, echo=echo, modality=modality)
                    if osp.isfile(json_filename):
                        with open(json_filename) as jsonFile:
                            data = json.load(jsonFile)
                            slice_times = data["SliceTiming"]
                        slice_times = ' '.join(str(e) for e in slice_times)
                        print(slice_times)
                        command="3drefit -Tslices "+slice_times+" "+nii_filename
                        # print(command)
                        os.system(command)

++ 3drefit: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-1_part-mag_sbref.nii.gz
*+ WARNING: -Tslices: dataset has no time axis to add slice offset to!
 + Didn't make any changes for dataset /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-1_part-mag_sbref.nii.gz !
++ 3drefit processed 1 datasets
++ 3drefit: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-2_part-mag_sbref.nii.gz
*+ WARNING: -Tslices: dataset has no time axis to add slice offset to!
 + Didn't make any changes for dataset /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-2_part-mag_sbref.nii.gz !
++ 3drefit processed 1 dataset

0 0.165 0.33 0.495 0.6575 0.8225 0.9875 1.1525 1.3175 1.48 1.645 1.81 1.975 2.1375 2.3025 2.4675 2.6325 2.7975 2.96 3.125 3.29 3.455 3.6175 3.7825 3.9475 4.1125 4.2775 4.44 4.605 4.77 4.935 5.0975 5.2625 5.4275 5.5925 5.755 5.92 6.085 6.25 6.415 6.5775 6.7425 6.9075 7.0725 7.235
0 0.165 0.33 0.495 0.6575 0.8225 0.9875 1.1525 1.3175 1.48 1.645 1.81 1.975 2.1375 2.3025 2.4675 2.6325 2.7975 2.96 3.125 3.29 3.455 3.6175 3.7825 3.9475 4.1125 4.2775 4.44 4.605 4.77 4.935 5.0975 5.2625 5.4275 5.5925 5.755 5.92 6.085 6.25 6.415 6.5775 6.7425 6.9075 7.0725 7.235
0 0.165 0.33 0.495 0.6575 0.8225 0.9875 1.1525 1.3175 1.48 1.645 1.81 1.975 2.1375 2.3025 2.4675 2.6325 2.7975 2.96 3.125 3.29 3.455 3.6175 3.7825 3.9475 4.1125 4.2775 4.44 4.605 4.77 4.935 5.0975 5.2625 5.4275 5.5925 5.755 5.92 6.085 6.25 6.415 6.5775 6.7425 6.9075 7.0725 7.235
0 0.165 0.33 0.495 0.6575 0.8225 0.9875 1.1525 1.3175 1.48 1.645 1.81 1.975 2.1375 2.3025 2.4675 2.6325 2.7975 2.96 3.125 3.29 3.455 3.6175 3.7825 3.9475 4.1125

++ 3drefit processed 1 datasets
++ 3drefit: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-2_part-mag_bold.nii.gz
 + setting slice time offsets
 + loading and re-writing dataset /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-2_part-mag_bold.nii.gz (/media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-2_part-mag_bold.nii.gz in NIFTI storage)


0 0.8225 0.165 0.9875 0.33 1.1525 0.495 1.3175 0.6575 0 0.8225 0.165 0.9875 0.33 1.1525 0.495 1.3175 0.6575 0 0.8225 0.165 0.9875 0.33 1.1525 0.495 1.3175 0.6575 0 0.8225 0.165 0.9875 0.33 1.1525 0.495 1.3175 0.6575 0 0.8225 0.165 0.9875 0.33 1.1525 0.495 1.3175 0.6575


++ 3drefit processed 1 datasets
++ 3drefit: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-3_part-mag_bold.nii.gz
 + setting slice time offsets
 + loading and re-writing dataset /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-3_part-mag_bold.nii.gz (/media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-00_echo-3_part-mag_bold.nii.gz in NIFTI storage)


...

++ 3drefit processed 1 datasets
++ 3drefit: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-13_echo-4_part-phase_bold.nii.gz
 + setting slice time offsets
 + loading and re-writing dataset /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-13_echo-4_part-phase_bold.nii.gz (/media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-13_echo-4_part-phase_bold.nii.gz in NIFTI storage)


0 0.82 0.1625 0.985 0.3275 1.15 0.4925 1.315 0.6575 0 0.82 0.1625 0.985 0.3275 1.15 0.4925 1.315 0.6575 0 0.82 0.1625 0.985 0.3275 1.15 0.4925 1.315 0.6575 0 0.82 0.1625 0.985 0.3275 1.15 0.4925 1.315 0.6575 0 0.82 0.1625 0.985 0.3275 1.15 0.4925 1.315 0.6575


++ 3drefit processed 1 datasets
++ 3drefit: AFNI version=AFNI_24.2.00 (Jul 15 2024) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-13_echo-5_part-phase_bold.nii.gz
 + setting slice time offsets
 + loading and re-writing dataset /media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-13_echo-5_part-phase_bold.nii.gz (/media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii/sub-18/func/sub-18_task-geo_run-13_echo-5_part-phase_bold.nii.gz in NIFTI storage)


0 0.82 0.1625 0.985 0.3275 1.15 0.4925 1.315 0.6575 0 0.82 0.1625 0.985 0.3275 1.15 0.4925 1.315 0.6575 0 0.82 0.1625 0.985 0.3275 1.15 0.4925 1.315 0.6575 0 0.82 0.1625 0.985 0.3275 1.15 0.4925 1.315 0.6575 0 0.82 0.1625 0.985 0.3275 1.15 0.4925 1.315 0.6575


++ 3drefit processed 1 datasets


### Check the slice timing results

In [20]:
%%bash

subject="sub-18"

fMRI_data_dir="/media/brain/Spatial/spatialfMRI/fMRI_data/spatialfMRI_nii"

3dinfo -slice_timing ${fMRI_data_dir}/${subject}/func/${subject}_task-geo_run-00_echo-1_part-mag_bold.nii.gz

0.000000|0.822500|0.165000|0.987500|0.330000|1.152500|0.495000|1.317500|0.657500|0.000000|0.822500|0.165000|0.987500|0.330000|1.152500|0.495000|1.317500|0.657500|0.000000|0.822500|0.165000|0.987500|0.330000|1.152500|0.495000|1.317500|0.657500|0.000000|0.822500|0.165000|0.987500|0.330000|1.152500|0.495000|1.317500|0.657500|0.000000|0.822500|0.165000|0.987500|0.330000|1.152500|0.495000|1.317500|0.657500
